# <center>  Data Inversion for Aerosol Science and Technology </center> 

__Author:__ Markus Petters (mdpetter@ncsu.edu)

__Abstract__. Many aerosol measurement techniques produce raw measurement response functions that must be inverted to properly interpret the data. This tutorial will introduce common inversion approaches used in Aerosol Science & Technology. One often used technique is mobility classification of aerosol using electrical mobility analyzers. The tutorial will briefly introduce the aerosol sampling with mobility analyzers and provide hands-on examples for several instrument configurations. The examples are designed to demonstrate how the inversion of mobility analyzer response functions is critical to informing experimental design and data analysis. At the end of the session, tutorial participants will have a starting point to modify supplied computer code for use in their own research projects.

<div class="alert alert-success">
 
**By the end of this tutorial you will be able to**

- Create and manipulate size distributions in code.
- Represent cylindrical and radial DMAs with arbitrary dimensions in code.
- Compute the moments of output from a DMA operated at a single voltage.
- Compute the convolution matrix for DMAs with corresponding to a custom instrument.
- Evaluate how measurement noise influences the least-squares inversion scheme.
- Apply regularization to smooth the least-squares inversion results.
- Apply the L-curve to identify the optimal regularization parameter.
- Apply the code to invert a DMA response function loaded from a data file.
- Apply the code to invert size resolved CCN data loaded from a data file.

</div>


<div class="alert alert-danger">

**Instructions**
    
- Go to Menu "Cell" -> "Run all Cells Below"
- The notebook will initialize all cells (~30 s)
- Navigate back to this point and start with the first video.    
- Work through the notebook sequentially.
 
</div>

# Welcome

In [ ]:
include("Scripts/startup.jl")
display("https://www.youtube.com/watch?v=bVeG1FrRUIE")

## Navigation and Symbols

In [ ]:
play("Audio/transfer-function1.ogg")

<div class="alert alert-success">

**You met the following learning objective**

- Create and delete cells in Jupyter.

</div>

<div class="alert alert-warning">

**This is a key point**
    
Newton's law will do the trick: $F = ma$

</div>

<div class="alert alert-danger">

**Important additional information**
    
- You need some calculus to find the velocity from $F = ma$.
- Further details are in the Feynman Lectures on Physics.

</div>

<div class="alert alert-info">

**Assignment**

Calculate the force a 1 kg heavy ball will exert on a scale.

</div>


## Jupyter Cells

In [ ]:
play("Audio/transfer-function1.ogg")

In [ ]:
println("Hello World")

<div class="alert alert-info">
Write one line of code to print "Aerosol Science Rocks!".
</div>

<div class="alert alert-info">
Create a new cell below this one calculate 1+1
</div>

## Outline 

<div class="alert alert-danger">

**This tutorial covers the following topics**

1. Introduction 
2. Size Distribution
3. Overview over the DMA
4. Transmission through the DMA at Constant Voltage
5. Inversion of Size Distribution Data
6. Inversion of Size-Resolved CCN Data
7. Summary and Perspective
8. Epilogue
    
</div>

# 1. Introduction

## 1.1 Inverse Problems

In [ ]:
play("Audio/transfer-function1.ogg")

**The inverse problem**

> [Source: Wolfram Mathworld](https://mathworld.wolfram.com/InverseProblem.html): To predict the result of a measurement requires (1) a model of the system under investigation, and (2) a physical theory linking the parameters of the model to the parameters being measured. This prediction of observations, given the values of the parameters defining the model constitutes the "normal problem," or, in the jargon of inverse problem theory, the forward problem. The "inverse problem" consists in using the results of actual observations to infer the values of the parameters characterizing the system under investigation. 


<div class="alert alert-warning">

**Approach to solve the inverse problem**

1. Write Fredholm integral equations of the first kind for specific problem
2. Discretize equation to write in Matrix form
3. Apply inversion with regularization with constraints
    
</div>

## 1.2 The Differential Mobility Analyzer

In [ ]:
play("Audio/transfer-function1.ogg")

The cylindrical DMA (Knutson and Whitby, 1975) passes a polydisperse aerosol flow through an annulus gap. An electric potential is applied between the inner and outer column. The electric potential drags charged particles through a viscous sheath flow. Particles whose mobility coincide with the electric and flow field forces will be steered to the sample slit. All other particles exit with the excess flow. 

<img src="Figures/nbs1_f01.png" width="400"> 
<b> Figure 1. </b> Schematic of a differential mobility analyzer. The cylindrical differential mobility analyzer is an annular capacitor. The column's properties are defined by the radii $r_1$, $r_2$, the length of the aerosol path, $l$. Operation conditions are defined by the the electric potential $v$ applied across the annulus and the four flow rates: sheath flow, $q_{sh}$, polydisperse aerosol flow $q_a$, excess flow, $q_{ex}$, and monodisperse sample flow, $q_{sa}$. Throughout this work it is assumed that the flows are balanced, i.e. $q_{sh} = q_{ex}$ and $q_{sa} = q_a$. The two flows tracked are $q_{sh}$ and $q_{sa}$. Dimensions and flow range for one commonly used model are provided in Table 1 below. <br>


<center>
<table style="width:45%">
    <caption> <b> Table. </b> Geometry of the TSI 3080 long cylindrical DMA column together with typical ranges for operation parameters. </caption>
  <tr>
    <th align="justify">Geometry parameter</th>
    <th align="justify">Value</th> 
    <th align="justify">Operational parameter</th>
    <th align="justify">Range</th> 
</tr>
  <tr>
    <td align="justify">$r_1$</td>
    <td align="justify">9.37×10⁻³ m</td> 
    <td align="justify">$v$</td>
    <td align="justify">± 10-10000 V</td> 
</tr>
  <tr>
    <td align="justify">$r_2$</td>
    <td align="justify">1.961×10⁻² m</td> 
    <td align="justify">$q_{sh}$</td>
    <td align="justify">2-20 L min⁻¹</td> 
</tr>
  <tr>
    <td align="justify">$l$</td>
    <td align="justify">0.44369 m</td> 
    <td align="justify">$q_{sa}$</td>
    <td align="justify">0.3-2 L min⁻¹</td> 
  </tr>
</table>
</center>


# 2. Size Distribution

## 2.1 Histogram Representation 

In [ ]:
play("Audio/transfer-function1.ogg")

In [ ]:
include("Scripts/size_distribution1.jl")

<b> Table 1. </b> Example representation of a lognormal aerosol size distribution on a 10-bin geometrically stepped grid ranging from 30 to 300 nm with geometric standard deviation 1.2 and total number concentration 200 cm⁻³ . Column 1 gives the bin number, Dlow is the lower bin diameter, Dup is the upper bin diameter, ΔlnD = ln(Dup) - ln(Dlow), Dp = $\sqrt{Dup \cdot Dlow}$ is the geometric midpoint diameter of the bin, S $\equiv \frac{dN}{d\ln Dp}$ is the number spectral density [cm⁻³] and N = $\frac{dN}{d\ln Dp} \cdot d\ln Dp$ is the number concentration in each bin [cm⁻³]. The total number concentration is recovered by summing over all bins, i.e. Nt = $\sum_{i=1}^{10} N$. For a geometrically stepped grid, the ΔlnD is identical in each bin. Numbers are rounded for clarity. 

#### Size Distribution Function

The multi-modal lognormal size distribution is given by (e.g. Seinfeld and Pandis, 2006)
<center> $\frac{dN}{d\ln D_p} = \sum_{i=1}^n \frac{N_{t,i}}{\sqrt{2\pi}\ln\sigma_{g,i}} \exp \left(- \frac{\left[\ln D_p-\ln D_{pg,i}\right]^2}{2\ln \sigma_{g,i}^2}\right) $  </center>
where $\frac{dN}{d\ln D_p}$ is the spectral number density, $N_{t,i}$ is the total number concentration, $\sigma_{g,i}$ is the geometric standard deviation, and $D_{pg,i}$ is the geometric mean diameter of the $i^{th}$ mode, and $n$ is the number of modes. <br> 

## 2.2 Representing the Size Distribution in Code

In [ ]:
play("Audio/transfer-function1.ogg")

The size distribution is a composite data type (structure) that has the fields Dp, De, ΔlnD, S, and N, corresponding to the table representation above.

<div class="alert alert-warning">
    
- 𝕟.Dp: Geometric midpoint diameters
- 𝕟.De: Bin edge diameters
- 𝕟.ΔlnD: Log bin spacing, ΔlnD = ln(Dup/Dlow)
- 𝕟.S: Spectral density
- 𝕟.N: Number concentration in the bin

</div>

In [ ]:
# This creates a size distribution with 10 bins between 30 and 300 nm. 
# It is the same size distribution as shown in Table 1.
𝕟 = lognormal([[200, 80, 1.2]]; d1 = 30.0, d2 = 300.0, bins = 10);

In [ ]:
# This is the array of bin edge diameters. Compare to Dlow and Dup in Table 1.
𝕟.De

<div class="alert alert-info">
Write one line of code to display the number concentration in each bin
</div>

<div class="alert alert-info">
Write one line of code to display the midpoint diameters
</div>

<div class="alert alert-info">
Write one line of code to extract the number concentration in the 5th bin. Julia starts indexing at 1. Square brackets are used to extract arrays, i.e. x[2] extracts the second element of array x
</div>

<div class="alert alert-info">
Write one line of code to create a size distribution from 10 to 1000 nm with 256 size bins. Call this size distribution myN.
</div>

## 2.3 Size Distribution Arithmetic

In [ ]:
play("Audio/transfer-function1.ogg")

### 2.3.1 Addition of two size distributions
Let 𝕟₁ and 𝕟₂ denote a two size distribution defined on the same diameter grid. Then
```julia
𝕩 = 𝕟₁ + 𝕟₂ 
```
is defined such that 
```julia
𝕩.S = 𝕟₁.S + 𝕟₂.S 
𝕩.N = 𝕩.S .* 𝕟.ΔlnD 
```

<div class="alert alert-warning">
The net result is the superposition of size distributions.
</div>

<div class="alert alert-danger">
The bins don't have be identical. The summation will rebin 𝕟₂ onto grid 𝕟₁
</div>

In [ ]:
# Example addition of size distributions
𝕟₁ = lognormal([[120, 90, 1.20]]; d1 = 10.0, d2 = 1000.0, bins = 256);  # size distribution
𝕟₂ = lognormal([[90, 140, 1.15]]; d1 = 20.0, d2 = 800.0, bins = 256);    # size distribution
𝕩 = 𝕟₁+𝕟₂
include("Scripts/pretty_plot_sizedistribution1.jl")

<b> Figure. </b> Illustration of the operation 𝕟₁+𝕟₂. The operation intuitively superimposed the two distributions into a single distribution. 

### 2.3.2 Multiplication of vector and size distribution
Let T denote a 1D vector that has the same number of elements as the size distribution $𝕟$. Then
```julia
𝕩 = T * 𝕟 
```
is defined such that 
```julia
𝕩.N = T * 𝕟.N
𝕩.S = T * 𝕟.S
```


<div class="alert alert-warning">
The net result is a bin-by-bin scaling of the number concentration.                               
</div>


In [ ]:
𝕟 = lognormal([[100, 100, 1.4]]; d1 = 10.0, d2 = 1000.0, bins = 128)  
μ,σ = 100.0, 200.0
T(x) = @. 0.5*(1.0 + erf((x - μ)/(sqrt(2σ)))) # Simple error function with mean μ and std. dev σ
𝕩 = T(𝕟.Dp)*𝕟    
include("Scripts/pretty_plot_sizedistribution2.jl")

<b> Figure. </b> Illustration of the operation T$*$𝕟. The array T is defined for each diameter point of the distribution. The convolution 𝕩 = T$*$𝕟 also applies to number concentration (not shown).

### 2.3.3 Multiplication of a scalar and a size distribution

Let a denote a floating point scalar and $𝕟$ denote a size distribution. Then
```julia
𝕩 = a⋅𝕟
```
is defined such that 
```julia
𝕩.Dp = a*𝕟.Dp 
```

<div class="alert alert-warning">
The net result is a uniform diameter shift of the size distribution.
</div>


In [ ]:
a = 2.0 # Note that a must be a floating point number
𝕟 = lognormal([[300, 100, 1.3]]; d1 = 10.0, d2 = 1000.0, bins = 256);  # size distribution
𝕩 = a⋅𝕟 
include("Scripts/pretty_plot_sizedistribution3.jl")

<b> Figure. </b> Illustration of the operation a⋅𝕟. The operation shifts the size distribution by a factor of two. Note, that the underlying diameter grid remains unchanged. This is achieved by interpolating the shifted size distribution back onto the original diameter grid. For this reason, the final fields dp and de are unaltered. 

### 2.3.4 Multiplication of a matrix and a size distribution
Let <b>A</b> denote a $n\times n$ matrix where $n$ equals the number of size bins of $𝕟$. Then
```julia
𝕩 = 𝐀*𝕟 
```
is defined such that 
```julia
𝕩.N = 𝐀*𝕟.N
𝕩.S = 𝐀*𝕟.S
```
The operator $*$ denotes matrix multiplication. 

<div class="alert alert-warning">
The net result is product of matrix 𝐀 and the concentration vector N.
</div>

<div class="alert alert-success">

**You have met the following learning objective:**
    
- Create and manipulate size distributions in code.
    
</div>

# 3. Representing DMAs

## 3.1 Representing DMA Geometry in Code

In [ ]:
play("Audio/transfer-function1.ogg")

The data type Λ defines the DMA in terms of flow rates and geometry and power supply polarity. The geometry parameters used in this example correspond to the TSI 3080 long column. The effective diffusion length describes diffusional loss in the DMA column (Reineking & Porstendörfer, 1986) which is described further below. Diffusional loss is ignored if leff = 0. Once instantiated Λ is queried to derive DMA/configuration specific properties.

In [ ]:
using DifferentialMobilityAnalyzers              # Load the package (only needed once)

qsa,qsh = 1.66e-5, 8.33e-5                       # Qsample [m3 s-1], Qsheath [m3 s-1]
t,p = 295.15, 1e5                                # Temperature [K], Pressure [Pa]
r₁,r₂,l = 9.37e-3,1.961e-2,0.44369               # DMA geometry [m]
leff = 13.0                                      # DMA effective diffusion length [m]
m = 6                                            # Upper number of charges
DMAtype = :cylindrical                           # specify DMA type as cylindrical or radial
polarity = :-                                    # negative :- or positive :+ polartiy

Λ = DMAconfig(t,p,qsa,qsh,r₁,r₂,l,leff,polarity,m,DMAtype)  

<div class="alert alert-info">
Write code in the cell below to instantiate a DMA with the following conditions: 
    
- positive polarity power supply
- room temperature
- Boulder pressure (840 hPa)
- cylindrical geometery
- no diffusional loss
- sheath-to-sample flow rate of 10:1
- sample flow rate 1 L min-1
- geometery of the TSI long column DMA
    
Name this DMA myΛ
</div>

## 3.2 Relationship between voltage, mobility, and diameter


In [ ]:
play("Audio/transfer-function1.ogg")

For the cylindrical DMA and balanced flows, the relationship between voltage, mobility, and diameter is given by Knutson and Whitby (1975)
<div class="alert alert-warning">
<center> $z^s = \frac{q_{sh}}{2\pi l v} \ln \left(\frac{r_2}{r_1}\right)$ </center>
where $v$ is the potential applied between the inner and out section of the annulus, $r_1$, $r_2$, and $l$ are the dimensions of the cylindrical DMA (Table 1) and $q_{sh}$ is the sheath flow rate. The relationship between diameter $d_p$ and centroid mobility "z star" ($z^s$) is 
<center> $d_p =  \frac{kec_c}{3\pi \eta z^s}$ </center>
where $e$ is the elementary charge and $k$ is the number of charges on the particle, $c_c$ is the Cunningham correction factor, and $\eta$ is the viscosity of the fluid. <br> 
</div>
    
Some of the conversions requires iteration. The function 

- ```vtod(Λ,v)``` converts voltage to diameter
- ```dtoz(Λ,d)``` converts mobility diameter to mobility
- ```ztod(Λ,k,z)``` converts mobility to mobility diameter for particles carrying k charges

and Λ subsumes the DMA geometery (r1,r2,l), sheath flow rate, and t,p.

In [ ]:
k = 1
v = 1000
z = vtoz(Λ, v)
dp = ztod(Λ, k, z)
@printf("V = %i V corresponds to Z = %.2e []\n", v, z)
@printf("Z = %.2e [] and n = %i charge(s) corresponds to Dp = %i nm\n", z, k, dp)
@printf("Dp = %i nm and n = 1 charge corresponds to Zp = %.2e nm\n", dp, dtoz(Λ,dp*1e-9))

<div class="alert alert-info">

Write code in the cell below to compute the mobility diameter of particles carrying two negative charges that would be selected at 5 kV potential by the DMA you configured above (myΛ).

</div>

<div class="alert alert-info">

Use the function ```dtoz(Λ,d)``` to verify that the diameter 114 nm corresponds to electrical mobility 2.20e-08 m2 V-1 s-1 for the DMA Λ, as claimed above. Note that d in dtoz is in units of m.

</div>

## 3.3 Representing the DMA Grid

In [ ]:
play("Audio/transfer-function1.ogg")

The DMA is operature between an upper and lower voltage limit. The full range is usually 10V to 10kV. At higher voltages the electric field breaks down. A convenient way to bin the DMA is to work with a log spaced mobility grid, which in essence is the size distribution histogram where diameters are computed using ztod. In code the DMA grid is represented using the DifferentialMobilityAnalyzer composite data type.

In [ ]:
bins,z₁,z₂ = 60, vtoz(Λ,10000), vtoz(Λ,10)    # bins, upper, lower mobility limit
δ = setupDMA(Λ, z₁, z₂, bins);                # Setup DMA grid

The ```δ``` structure that contains 

- ```Z``` are the mobility bin midpoints, 
- ```Dp``` are the diameter bin midpoints (internally in units of nm) 
- ```Ze``` are the mobility bin edges, 
- ```De``` are the diameter bin edges, 
- ```ΔlnD``` is the natural log ratio of upper and lower diameter bin edge

In [ ]:
# Above we generated 60 bins between 10V and 10 kV for DMA Λ. Thise are the midpoint diameter bins.
δ.Dp

<div class="alert alert-info">

Define your DMA grid myδ for your DMA myΛ, using a 30 bin representation and assuming that the DMA is operated between 50V and 5kV. Show the diameter grid.

</div>

<div class="alert alert-info">

Write a statement that shows the mobility grid corresponding to you diameter grid

</div>

<div class="alert alert-success">

**You have met the following learning objective**

- Represent cylindrical and radial DMAs with arbitrary dimensions in code.

</div>

## 3.4 (Diffusing) DMA Transfer Function

In [ ]:
play("Audio/transfer-function1.ogg")

In [ ]:
include("Scripts/plot_transfer_function1.jl")

**Figure.** Normalized DMA transfer function for DMA Λ with sheath-to-sample flow ratio β = 1/5  and 20 and 200 nm particles. The 20 nm transfer function is smeared due to diffusional broadening.

The DMA transfer function is the probability that a particle of a particle of a given size exits the classifier via the sample flow. The diffusive broadened DMA transfer function is computed assuming blanced sheath and excess flows using the expression of Stolzenburg and McMurry (2008).
<center> $ \Omega(\tilde{z},\beta,\sigma) = \frac{\sigma}{\sqrt{2}\beta}\left[\epsilon \left( \frac{\tilde{z}-(1+\beta)}{\sqrt{2}\sigma} \right) + \epsilon \left (\frac{\tilde{z}-(1-\beta)}{\sqrt{2}\sigma} \right) - 2\epsilon \left ( \frac{\tilde{z}-1}{\sqrt{2}\sigma}\right)  \right]$ </center>
    
where $\tilde{z} = \frac{z}{z^s}$ is the dimensionless mobility, $z$ is the particle mobility $z^s$ is the centroid mobility selected by the DMA, $\epsilon = x \mathrm{erf}(x) +\left(\exp(-x^2)/\sqrt{\pi}\right)$, $\mathrm{erf}$ is the error function, and $\beta = \frac{q_{sa}}{q_{sh}}$. The parameter $\sigma$ accounts for diffusional broading of the transfer function. Assuming plug flow, $\sigma$ can be computed using the following equations Hagwood (1999) <br> <br>
$\gamma = \left(\frac{r_1}{r_2}\right)^2$ <br>
$I = \frac{1}{2}(1+γ)$ <br>
$\kappa = \frac{lr_2}{r_2^2-r_1^2}$ <br>
$G = \frac{4(1+\beta)^2}{(1-γ)} \left[I+\{2(1+\beta)\kappa\}^{-2} \right ] $<br>
$\sigma = \sqrt{\frac{2G\pi ld_{ab}}{q_{sh}}}$ <br> <br>

In [ ]:
play("Audio/transfer-function2.ogg")

<div class="alert alert-warning">
The transfer function is included in the "DMA grid" δ

- ```δ.Ω(Λ,z,zˢ)``` where z is a vector of mobilities and zˢ ("z star") is the centroid mobility. 	
</div>

In [ ]:
zˢ = dtoz(Λ, 200e-9)      # centroid mobility for Dp = 200 nm
z = δ.Z                   # mobility grid defined for the DMA (60 bins above)
δ.Ω(Λ,z,zˢ)               # Output of the transfer function
DataFrame(z = z, Ω = δ.Ω(Λ,z,zˢ))  # To make a pretty table

In [ ]:
play("Audio/transfer-function3.ogg")

In [ ]:
include("Scripts/plot_transfer_function2.jl")

**Figure.** Example SMPS transfer function Ωav.

<div class="alert alert-warning">
    
During operation as scanning mobility particles, the voltage continously changes. Signal is acquired during some discrete time interval $t_c$. The SMPS transfer function is calculated as the average transfer function during the time interval $[t,t+t_c]$ (Wang and Flagan, 1990). <br> <br>

<center> $\Omega_{av} = \frac{1}{tc}\int_{t_i}^{t_i+t_c} \Omega(Z,z^s(t)) dt$ </center>

where $t_i$ is the start time when counting begins in channel $i$, $z^s(t)$ is the selected centroid mobility at time t and is calculated from the applied voltage. The voltage is usually varied exponentially with time from high-to-low (downscan) or low-high (upscan). The average transfer function is obtained by numerical integration.
</div>

## 3.5 Equilibrium Charge Distribution Function

In [ ]:
play("Audio/transfer-function1.ogg")

In [ ]:
include("Scripts/pretty_plot_charge_fraction.jl")

Charging efficiency (charge equilibrium) obtained in the bipolar charger is computed based on the parameterized measurements by Wiedensohler et al. (1988) with coefficients taken from the TSI 3080 Manual (2009). 
<center> $T_c(k) = 10^\left\{ \sum_{i=1}^6 a_i (k) \left[ \ln \left(\frac{D_p}{nm}\right) \right]^{i-1} \right \}$ </center>
where $k = -2,-1,1,2$ is the number and polarity of particle charge and $a_i$ are empirical coefficients. <br> <br>
The function Tc(k) is defined in module DifferentialMobilityAnalyzer.jl. It return the charging efficiency for n charges. Each function belongs to a DMA structure, which sets the internal diameter grid and polarity. In this script, the function δ.Tc is for negative polarity and δp.Tc for positive polarity as defined above. The function is computes charging efficiencies on the DMA internal grid δ.dp, which is set by the mobility range and the number bins.

For $k \ge \pm 3$, the formula from the TSI manual is used:
<center> $T_c(k) = \frac{e}{\sqrt{4\pi^2\epsilon D_pk_bT}} \exp \left( \frac{-\frac{\left[|k| - 2\pi\epsilon D_pk_bT \ln(0.875)\right]^2}{e^2}}{ \frac{4\pi\epsilon D_pk_bT}{e^2}} \right) $</center>
where $e$ is the elementary charge and $\epsilon$ is the dielectric constant for air.

<div class="alert alert-warning">
The charge efficiency function is included in the "DMA grid" δ

- ```δ.Tc(k,dp)``` where k is the number of charges and dp is the diameter in units of nm. 	
</div>

In [ ]:
# The fraction of charges carrying 1 charge. The polarity is defined by the DMA Λ that was used to 
# setup the grid δ 
δ.Tc(1,100.0) 

<div class="alert alert-info">

Write a one line expression that computes the fraction of particles carrying 3 charges  along the internal diameter grid of your DMA myδ

</div>

<b> Figure. </b> Size dependence of fractional charging efficiency of through the bipolar charger for 1-4 charges.

## 3.6 Particle Losses/Detector Efficiency

In [ ]:
play("Audio/transfer-function1.ogg")

Some fraction of particles is lost during transit. Particles may not be detected at 100% efficiency. These losses can be described using a single loss function (or a series of loss functions). Here we consider just the penetration efficiency through the DMA as computed based on the parameterization of  Reineking & Porstendörfer (1986).

<div class="alert alert-warning">
The loss function is Tl included in the "DMA grid" δ

- ```δ.Tl(Λ,dp)``` where Λ is the DMA configuration (flow rate dependency and effective length) and dp is the diameter in units of nm. 	
</div>

In [ ]:
include("Scripts/pretty_plot_loss_fraction.jl")

<b> Figure. </b> Size dependence of penetration efficiency through the DMA for leff = 13 m and two flow rates.

## 4. Transmission through the DMA at Constant Voltage

In [ ]:
play("Audio/transfer-function1.ogg")

The Figure shows a schematic instrument setup with an array of detectors. The DMA is operated in classifier mode and set to a single voltage. The mobility classified particles are then passed to one or more detectors. 
<img src="Figures/nbs2_f01.png" width="350" align = "middle">
<figcaption><b> Figure. </b> Illustration of the combined transfer function through the DMA. </figcaption> 


<div class="alert alert-warning">
The net transmission function T describes transmission through the DMA, including the charge filter, the DMA transfer function, and the DMA transmission efficiency. 

```julia
T(zˢ,k,Λ,δ) = δ.Ω(Λ,δ.Z,zˢ/k).*δ.Tc(k,δ.Dp).*δ.Tl(Λ,δ.Dp)
```

where Λ is the DMA configuration, δ the DMA grid, zˢ is the z-star selected by the DMA, and k is the number of charges. 	
</div>



In [ ]:
# Define the mobility
zˢ = dtoz(Λ, 100*1e-9)     

# Define the DMA transmission function
T(zˢ,k,Λ,δ) = δ.Ω(Λ,δ.Z,zˢ/k).*δ.Tc(k,δ.Dp).*δ.Tl(Λ,δ.Dp)

# Plot the result
plot(x = δ.Dp, y = T(zˢ,1,Λ,δ), Geom.step, Guide.xlabel("Apparent + 1 Mobility Diameter (nm)"), Guide.ylabel("Fraction (-)"))

**Figure.** The shape is the transfer function, modulated by losses and charging efficiency and represented at the bin resulution of the DMA grid.

The transmission function ```T(zˢ,k,Λ,δ)``` represents a vector along the DMA grid that predicts the fraction of particles carrying k charges that transmit if the DMA voltage is set to selected zˢ. It is therefore a vector that is defined along the DMA mobility/diameter grid. For our DMA defined above we had 60 bins.

In [ ]:
T(zˢ,1,Λ,δ)

## 4.1 Output Distribution from a DMA

In [ ]:
play("Audio/transfer-function1.ogg")

In [ ]:
# Let's make a high resultion grid for visualization purposes (this is slow!))
bins,z₁,z₂ = 512, vtoz(Λ,10000), vtoz(Λ,10)    # bins, upper, lower mobility limit
δ1 = setupDMA(Λ, z₁, z₂, bins);                # Setup DMA grid

<div class="alert alert-warning">
    
- zˢ is the z-star selected by the DMA
- 𝕟ᶜⁿ is an assumed known bimodal lognormal distribution computed on the DMA size grid
- ℕ is an array of transmitted mobility distributions carrying k charges
- 𝕄 is an array of transmitted apparent mobility distributions carrying k charges
- 𝕟ₜ, 𝕞ₜ are the superposition of these distributions
</div>


In [ ]:
zˢ = dtoz(Λ, 100*1e-9)                                                      # Mobility for 100 nm paticles
𝕟ᶜⁿ = DMALognormalDistribution([[900., 40., 1.5], [500., 180., 1.4]], δ1);  # Returns the size distribution 
ℕ = map(k -> T(zˢ,k,Λ,δ1)*𝕟ᶜⁿ,1:3)                                  # ℕ[k] : Mob. size distribution +k charges
𝕄 = map(k -> (ztod(Λ,1,zˢ)/ztod(Λ,k,zˢ))⋅(T(zˢ,k,Λ,δ1)*𝕟ᶜⁿ),1:3)   # 𝕄[k] : App. Mob. distribution +k charges 
𝕟ₜ, 𝕞ₜ = sum(ℕ), sum(𝕄)

include("Scripts/pretty_plot_transmission.jl")

<b> Figure. </b> Left: assumed bimodal lognormal size distribution. Middle: monodisperse mobility size distribution plotted against the apparent +1 mobility diameter, defined as the apparent setpoint diameter of the DMA. Dashed line is total number concentration. Right: same as middle panel but plotted
versus the mobility diameter.


The DMA selects approximately a triangular distribution around mobility centroid $z^s$. The distribution is symmetric when plotted against the log of mobility, but asymmetric when plotted against the log of the apparent +1 mobility diameter because the Cunningham slip flow correction factor applied in the conversion from mobility to diameter is a strong function of particle size. The majority of selected particles are singly charged, but the contribution of multiply charged particles to the total number is not negligible. The mobility distribution has contributions from particles that are at least twice the diameter of the selected centroid diameter. The relative fractions are determined by the equilibrium charge fraction and the number of particles available at each diameter.


<div class="alert alert-danger">
    
The ```map(k->f, 1:3)``` construct sequentially applies values from the array [1,2,3] to k and calls the function f. The output is an array of length 3. Since ```T(zˢ,k,Λ,δ)``` produces a vector, 𝕟ᶜⁿ is a size distribution, and vector * size distribution is a size distribution, the output of
    
```
ℕ = map(k -> T(zˢ,k,Λ,δ)*𝕟ᶜⁿ,1:3) 
```
    
is an array of size distributions.
</div>

<div class="alert alert-info">

1. Predict how the number concentration of 𝕄[3] and ℕ[3] will change if the number concentration of the accumulation mode were to decrease, i.e. change from 500 cm-3 to 5 cm-3.

2. Change the number concentration in the 
    ```𝕟ᶜⁿ = DMALognormalDistribution([[900., 40., 1.5], [500., 180., 1.4]], δ)```
    statement in the cell above and rerun the cell with CTRL-Enter. Compare with your prediction.

</div>

## 4.2 Example Application: Calibrate a Particle into Liquid Sampler

In [ ]:
play("Audio/transfer-function1.ogg")

### Number Concentration Measured by the CPC

The number concentration reported by the CPC is the integral over the DMA output size distribution 𝕟ₜ. Since the size distribution data type includes the number concentration in each bin, it is simply the sum of number concentration is each bin

```julia
Ncpc = sum(𝕟ₜ.N)
```

Alternatively the number concentration can be computed from the individual distributions ℕ[1], ℕ[2], ℕ[3] and so forth. This can be accomplished using a simple function.

```julia
Number(ℕ) = sum(map(𝕟 -> sum(𝕟.N), ℕ))
```

In [ ]:
Number(ℕ) = sum(map(𝕟 -> sum(𝕟.N), ℕ))
Ncpc1 = sum(𝕟ₜ.N)
Ncpc2 = Number(ℕ)
println(Ncpc1, "                 ", Ncpc2)  # Show that the two methods produce the same result are the same

### Naive Mass Estimate

The DMA is set to select a single apparent +1 mobility size, e.g. 100 nm in the example above. If we ignore the contribution of multiply charged particles, the mass selected is 

$M=\frac{\pi}{6}D_{p}^{3}N\rho_{p}$

where $\rho_{p}$ is the particle density. 

<div class="alert alert-warning">
Note that for mass calculations, if the density is $1000\; kg\;m^{-3} = 1\; g\; cm^{-3}$, the volume concentration in $\mu m^{3}\;cm^{-3}$ equals to the mass concentration in $\mu g\;m^{-3}$. This is an important conversion and explains why $\mu m^{3}\;cm^{-3}$ is the preferred unit for volume concentration.
</div>

In [ ]:
NaiveMass(Dp, ρp) = pi/6*Dp^3.0*sum(𝕟ₜ.N)  # Diameter in micron, and density in g cm-3
NaiveMass(0.1, 1.72)                       # Naive mass concentration in mug m-3

### Mass Accounting for Multiply Charged Particles

To compute the correct mass we must integrate over the distributions ℕ[1], ℕ[2], ℕ[3] and so forth, as we did with number, and weigh them by Dp^3. Because of the Dp^3 weighing the contribution of +2 and +3 charged particles to selected mass can be substantial. Depending on the shape of the size distribution

In [ ]:
# Diameter in micron, and density in g cm-3
Mass(ℕ, ρp) = sum(map(𝕟 -> sum(pi ./ 6 .* (𝕟.Dp .* 1e-3).^3.0 .* 𝕟.N .* ρp), ℕ))

println("Naive mass concentration   ", NaiveMass(0.1, 1.72))
println("Correct mass concentration  ", Mass(ℕ, 1.72))


<div class="alert alert-info">

1. Predict how the ratio of naive mass concentration to correct mass concentration will change if the number concentration of the accumulation mode were to decrease, i.e. change from 500 cm-3 to 5 cm-3. (Same exercise as previously.)

2. Change the number concentration in the 
    ```𝕟ᶜⁿ = DMALognormalDistribution([[900., 40., 1.5], [500., 180., 1.4]], δ)```
statement in the cell above and rerun all of the cells with CTRL-Enter. Compare with your prediction.

</div>

<div class="alert alert-success">

**You have met the following learning objective:**
    
- Compute the moments of output from a DMA operated at a single voltage.
    
</div>

## 4.3 The DMA Response Function (Fredholm Integral)

In [ ]:
play("Audio/transfer-function1.ogg")

As worked out above, the net transmission function T describes transmission through the DMA, including the charge filter, the DMA transfer function, and the DMA transmission efficiency. 

```julia
T(zˢ,k,Λ,δ) = δ.Ω(Λ,δ.Z,zˢ/k).*δ.Tc(k,δ.Dp).*δ.Tl(Λ,δ.Dp)
```

Evaluating for all charges and integrating over size yields the total number concentration measured by the CPC. 

```julia
ℕ = map(k -> T(zˢ,k,Λ,δ1)*𝕟ᶜⁿ,1:k)
𝕟ₜ = sum(ℕ)
```

Formally, the response in channel i (corresponding to a transmission through the DMA at a single voltage) is given by the Fredholm convolution integral 

 <center> $ R_i = \int_{z_a}^{z_b} \sum_{k=1}^n \Omega(z,Z_{i,k}^s)T_c(D[z,1])T_l(D[z,1])\frac{dN}{d\ln D}\frac{d \ln D}{dz}dz \;\;\;\;\;\;\ i = 1,2...,n$   </center>
 
The integral is performed over the limits $z_a$ and $z_b$, which corresponds to the upper and lower mobility limit set by the voltage range used to operate the DMA. The function $\frac{dN}{d\ln D}\frac{d\ln D}{dz}dz$ evaluates to the number concentration of particles that lie in the interval $[z,z + dz]$. Note that $D[z,1]$ is used in the charge filter and loss function since the integral is applied over the transform of the selected centroid mobility $Z_{i,k}^s$ (see above). $Z$ is the mobility vector of the grid and the subscript $i$ denotes the response channel. The convolution integral can be discretized:  <br><br>

 <center> $ R_i = \sum_{j=1}^n \left [ \sum_{k=1}^m \Omega(Z_j,Z_{i,k}^s)T_c(D[Z_j,1])T_l(D[Z_j,1])N(Z_j) \right] $</center> <br>

$N(Z_j)$ is the the number concentration of particles in the $j^{th}$ bin, $i = 1...n$
are indices the observed instrument channel, $j = 1...n$ are indices of the physical size bins, and $k = 1...m$ are indices of charges carried by the particle. Here it is assumed that $\Omega(Z_j,Z_{i,k}^s)$ can be approximated being constant over the bin $[Ze_{j},Ze_{j+1}]$, which is only true if a sufficiently large number of size bins is used. The discretized version represents a set of $n$ equations that can be written in matrix form <br><br>

<center> 
$ \begin{bmatrix}
       R_1    \\
       \vdots \\        
       R_n
\end{bmatrix} =
\begin{pmatrix}
  \sum_{k=1}^m \Omega(Z_1,Z_{1,k}^s)T_c(D[Z_1,1])T_l(D[Z_1,1])  & \cdots & \sum_{k=1}^m \Omega(Z_n,Z_{1,k}^s)T_c(D[Z_n,1])T_l(D[Z_n,1]) \\
  \vdots  & \ddots  & \vdots  \\
  \sum_{k=1}^m \Omega(Z_1,Z_{n,k}^s)T_c(D[Z_1,1])T_l(D[Z_1,1])  & \cdots & \sum_{k=1}^m \Omega(Z_n,Z_{n,k}^s)T_c(D[Z_n,1])T_l(D[Z_n,1])  
\end{pmatrix}
\begin{bmatrix}
       N_1    \\[0.3em]
       \vdots \\[0.3em]        
       N_n
\end{bmatrix}
$</center> <br><br>



<div class="alert alert-warning">

The discrete version of the Fredholm equation can thus be cast as the following
    
   <center> $R = \mathbf{A}N$ </center>

where $R$ is the response vector, $\mathbf{A}$ is the convolution matrix, and $N$ is the discretized true number concentration.
</div>

 


<div class="alert alert-danger">
    
    
Note how the terms comprising the convolution matrix 
    
   <center> $ \Omega(Z_1,Z_{1,k}^s)T_c(D[Z_1,1])T_l(D[Z_1,1]) $ </center>
    
are the transmission function defined in code
    
```julia
T(zˢ,k,Λ,δ) = δ.Ω(Λ,δ.Z,zˢ/k).*δ.Tc(k,δ.Dp).*δ.Tl(Λ,δ.Dp)
```
</div>

## 4.4 Computing the Convolution Integral

In [ ]:
play("Audio/transfer-function1.ogg")




<div class="alert alert-warning">
    
 The convolution integra can be constructed using the following expression. Admittedly, this code is a bit cryptic. Please see Notebook S2 in Petters (2018) if you are curious why this line solves the Fredholm equation. 

    
```julia
𝐀 = (hcat(map(zˢ->Σ(k->T(zˢ,k,Λ,δ),Λ.m),δ.Z)...))'
```
    
</div>

<div class="alert alert-danger">

Note that to evaluate this expression, the function T(zˢ,k,Λ,δ) has to be defined and the DMA has to be configured through Λ (flow rates, geometry) and δ (voltage range, number of bins). The matrix 𝐀 is n x n where n is the number of bins.
</div>

In [ ]:
# Here is how to compute the convolution matrix. The time to compute the matrix scales with the number of bins
T(zˢ,k,Λ,δ) = δ.Ω(Λ,δ.Z,zˢ/k).*δ.Tc(k,δ.Dp).*δ.Tl(Λ,δ.Dp)
𝐀 = (hcat(map(zˢ->Σ(k->T(zˢ,k,Λ,δ),Λ.m),δ.Z)...))'

<div class="alert alert-info">

Write and expression that computes the convolution matrix for your DMA, defined as myΛ and myδ

</div>

Note that you can not only define your own instrument configuration, but also change the convolution terms. For example, you can omit the charge filter, you can omit the transmission function through the DMA. In practice, these are just short computer functions and you can replace them with your own as needed. For example you swap out the transfer function to a custom version, or add a function that describes the detetector efficiency.

<div class="alert alert-info">

Write and expression that computes the convolution matrix for your DMA, defined as myΛ and myδ, but without accounting for transmission efficiency through the DMA.

</div>

<div class="alert alert-success">

**You have met the following learning objective:**
    
- Compute the convolution matrix for DMAs with corresponding to a custom instrument.
    
</div>

# 5. Inversion of Size Distribution Data

In [ ]:
play("Audio/transfer-function1.ogg")

The response function can be computed from the convolution matrix and the known true size distribution

<img src="Figures/nbs2_f02.png" width="750" align = "middle">
<b> Figure 1. </b> Convolution of true size distribution resulting in the response vector. <br> <br>

<div class="alert alert-warning">

In the section on size distribution arithmetic I included the operation matrix * size distribution. Thus we can conveniently write 

𝕣 = 𝐀*𝕟

to compute the response function
    
</div>

In [ ]:
# Here is a simple example
𝕟 = DMALognormalDistribution([[400, 30, 1.2],[500, 110, 1.7]], δ)
𝕣 = 𝐀 * 𝕟

In [ ]:
include("Scripts/pretty_plot_response_function.jl")

**Figure.** True size distribution and DMA response function.

## 5.1 Least-Squares Inversion

In [ ]:
play("Audio/transfer-function1.ogg")

<div class="alert alert-warning">

The response function is measured. The number concentration can be found through a matrix inversion <br><br>
    
    
<center> $ \rm{𝕟} = \bf{A}^{-1}\rm{𝕣}$ </center><br>
This simple inversion approach can be sufficient in some cases, However, even seemingly small noise  is amplified leading to unusable estimates of $\rm{𝕟}$ (e.g., Kandlikar and Ramachandran, 1999). 
    
</div>

In [ ]:
include("Scripts/noise_app.jl")

**Figure.** Influence of measurement noise on the least-squares solution.

<div class="alert alert-danger">
    
The solution is often dominated by contributions from data errors and rounding errors. Measurement noise is amplified in the inversion and the simple inverse produces unusable results. Measurement noise increases with increasing binsize and decreasing sample flow rate. The interactive applet demonstrates how these factors influence the inverted size distribution.
    
</div>

**Measurement noise:** Typically a condensation particle counter is used to measure the resonse function. Noise in the condensation particle counter is mostly from counting statistics. The Poisson counting statistic depends on the number of counts per bin, and thus the sample flow rate and integration time per bin. In the app below the scan time for the entire distribution is fixed to 2 min. Both the number of bins and the sample flow rate influence the measurement noise. "Noise free" CPC means that there is no counting error in the CPC.

<div class="alert alert-info">

1. Predict how changing total number concentration, CPC flow rate, and number of bins will influence the noise in the inverted size distribution.
    
2. Use the applet above to verify or falsify your prediction.

</div>

<div class="alert alert-success">
   
**You met the following learning objective**
    
- Evaluate how measurement noise influences the least-squares inversion scheme.
    
</div>

## 5.2 Tikhonov Regularization

In [ ]:
play("Audio/transfer-function1.ogg")

### 5.2.1 Least-Squares Inversion

Another way to look at the inversion described above is to cast the problem as a general least-squares problem

$L_1 = \left\lVert \bf{A}\rm{𝕟} - \rm{𝕣}\right\rVert_2$ <br>

where $L_1$ is the Euclidian norm of the residual. The optimal inverse solution is defined such that $L_1^2$ is minimized.

$𝕟_{inv} = \arg \min\{\left\rVert\bf{A}\rm{𝕟}-\rm{𝕣}\right\rVert_2^2$}

$ \rm{𝕟} = \bf{A}^{-1}\rm{𝕣}$ is the analytical solution that minimizes $L_1$. 

If the matrix $\bf{A}$ is rank deficient and not invertible the pseudo-inverse will yield the solution with the lowest $L_1$ norm

$\rm{𝕟} = \bf{({A^TA})^{-1}A^T} \rm{𝕣}$ 

<div class="alert alert-warning">

Noisy least-square inverse solutions have a large residual error and thus a large $L_1$ norm.
    
</div>

### 5.2.2 Regularization

To filter noise a regularization term is introduced

$L_2 = \left\lVert\bf{L}(\rm{𝕟} - \rm{𝕟_i})\right\rVert_2$

where $\bf{L}$ is a weights matrix and $\rm{𝕟_i}$ is an initial guess. The $L_2$ norm describes the deviation from a smooth initial guess. The inverse is a solution that balance between the $L_1$ and $L_2$ norms.

$𝕟_{inv} = \arg \min\{\left\rVert\bf{A}\rm{𝕟}-\rm{𝕣}\right\rVert_2^2 + \lambda^2 \left\rVert\bf{L}(\rm{𝕟}-\rm{𝕟_i})\right\rVert_2^2$

 $\lambda$ is the regularization parameter. Taking $\bf{L} = \bf{I}$ (weight matrix equals the identity matrix) and $\rm{𝕟_i} = \bf{S}^{-1}\rm{𝕣}$ as initial guess, the regularized inverse is computed via:

$𝕟_{inv} = (\bf{A}^\rm{T}\bf{A} + \lambda^\rm{2} \bf{I})^\rm{-1}(\bf{A}^\rm{T} \rm{𝕣} + \lambda^2\bf{S}^{-1} \rm{𝕣})$ <br>

This is also known as zeroth order Tikhonov regularization, or Phillips–Twomey regularization. 


<div class="alert alert-warning">

The regularization parameter $\lambda$ "interpolates" between the noisy least-square inverse ($\lambda = 0$) and the initial guess ($\lambda >> 1$) by placing weights on either the $L_1$ or the $L_2$ norm. 
    
</div>


<div class="alert alert-danger">
    
**To prove that** $𝕟_{inv} = (\bf{A}^\rm{T}\bf{A} + \lambda^\rm{2} \bf{I})^\rm{-1}(\bf{A}^\rm{T} \rm{𝕣} + \lambda^2\bf{S}^{-1} \rm{𝕣})$ **is the correct solution**
 
- Set $\bf{L} = \bf{I}$
- Evaluate $\frac{\partial}{\partial \rm{𝕟}}\left(\left\rVert\bf{A}\rm{𝕟}-\rm{𝕣}\right\rVert_2^2 + \lambda^2 \left\rVert\bf{I}(\rm{𝕟}-\rm{𝕟_i})\right\rVert_2^2\right) = 0$
- Use http://www.matrixcalculus.org/ to evaluate the derivative if you forgot your symbolic matrix calculus
- Substitute $\rm{𝕟_i} = \bf{S}^{-1}\rm{𝕣}$ 
- Solve for 𝕟
   
</div>


<div class="alert alert-info">

Show that for $\lambda = 0$, the Tikhonov inverse reduces to the regular-least-squares inverse: $\rm{𝕟} = \bf{({A^TA})^{-1}A^T} \rm{𝕣}$.
    
</div>


In [ ]:
play("Audio/transfer-function1.ogg")

In [ ]:
include("Scripts/regularization_app1.jl")

**Figure**. Left: Least-squares solution, Middle: Tikhonov with λ selected by the slider, Right: initial guess.

<div class="alert alert-warning">

- Talukdar and Swihart (2003) introduced a clever initial guess: sum the rows of 𝐀 and place the results on the diagonal of 𝐒. The least-squares inverse $\rm{𝕟_i} = \bf{S}^{-1}\rm{𝕣}$ gives excellent results where singly charged particles dominate and slight error where multiply charged particles become important.
- The constraint of the initial condition results in a robust inversion.
- The algorithm provides a reasonable inversion for a wide range of λ.
       
</div>

In [ ]:
include("Scripts/regularization_app2.jl")

**Figure**. Left: Least-squares solution, Middle: Tikhonov with λ selected by the slider, Right: initial guess.

<div class="alert alert-warning">

- Tikhonov can produce good results even in the absence of a good initial guess.
- However, the choice of λ is critical to provide the correct inversion.

</div>

In [ ]:
play("Audio/transfer-function1.ogg")

In [ ]:
# Regularization in code
λ, bins = 0.3, length(δ.Dp)

# some response distribtion
𝕣 = DMALognormalDistribution([[400, 30, 1.2],[500, 110, 1.7]], δ)

# Identity, Talukda and Swihart, and Convolution Matrices for DMA grid 
𝐈, 𝐒, 𝐀 =  δ.𝐈, δ.𝐒, δ.𝐀

@time 𝕟ⁱⁿᵛ = (𝐀'𝐀 + λ^2𝐈)^(-1) * (𝐀'𝕣  + λ^2 * 𝐒^(-1)*𝕣)

<div class="alert alert-danger">

- For convenience, the matrices 𝐈, 𝐒, 𝐀 are computed when the DMA grid is initialized. 
- They are stored in the δ structure together with the other grid variables.
       
</div>

<div class="alert alert-success">
   
**You met the following learning objective**
    
- Apply regularization to smooth the least-squares inversion results.
    
</div>

### 5.2.3 Optimal Regularization Parameter (L-curve Method)

In [ ]:
play("Audio/transfer-function1.ogg")

The optimal $\lambda_{opt}$ is found using the L-curve method. The L-curve is defined as a plot of $\log(L_1)$ vs.  $\log(L_2)$, where $L_1$ and $L_2$ are obtained for a series of discrete $\lambda$ values. The $\lambda_{opt}$ is found at the corner of the L-curve, which is mathematically defined as the point where where the curvature of the L-curve is maximum. Here, the corner of the L-curve is found using the iterative algorithm described in Talukdar and Swihart (2003). The curvature is calculated using Eq. (14) in Hansen (2000), which requires the first and second derivatives of $d\ln(L_i)^2/d\lambda$. These derivatives of $d\ln(L_i)^2/d\lambda$ are estimated numerically.

In [ ]:
include("Scripts/lcurve1.jl")

**Figure.** Left: Illustration of the L-curve. Right: Size distribution from inversion using the optimum regularization parameter. The raw distribution corresponds to N = 500 cm-3, Q = 1 L min-1 and 120 bins in the and is the same distribution as in the "regularization_app1". 

<div class="alert alert-warning">

- This is the L-curve for the case with strong constraints.
       
</div>

In [ ]:
include("Scripts/lcurve2.jl")

**Figure.** Left: Illustration of the L-curve. Right: Size distribution from inversion using the optimum regularization parameter. The raw distribution corresponds to N = 500 cm-3, Q = 1 L min-1 and 120 bins in the and is the same distribution as in the "regularization_app2". 

<div class="alert alert-warning">

- This is the L-curve for if the intial guess is zero.
- Tikhonov + L-curve still finds an acceptable solution, though the solution is noiser.
       
</div>

<div class="alert alert-success">
   
**You met the following learning objective**

- Apply the L-curve method to identify the optimal regularization parameter.

</div>

<div class="alert alert-danger">

- The L-curve search is slow due to the need to compute derivatives and rerun the matrix inversion many times. For this specific problem, the initial guess is very good. For "fast" inversion and using the initial guess, a characteristic regularization parameter ~0.2 to 0.6 can be used to give adequate results. The value to pick will depend on the CPC and DMA.
    
- A variety of methods are available to identify the optimal $\lambda$. These may be consulted if the L-curve minimum approach fails due to difficulty of estimating derivatives.
   
- Higher order Tikhonov solutions use estimates other then $\bf{L} = \bf{I}$.
    
- The following papers give excellent additional information about Tikhonov regularization: Twomey (1963), Kandlikar & Ramachandran (1999), Hansen (2000), Sipkens et al. (2020).
</div>

## 5.3 Example Inversion of Real Data

In [ ]:
play("Audio/transfer-function1.ogg")

In [2]:
using DifferentialMobilityAnalyzers, CSV,  DataFrames, Printf
# Load a simple comma delimited text file
df = CSV.read("example_data.csv")

,Dp,Rcn,Rccn
,Float64,Float64,Float64
1,11.23,2.16012,0.0
2,11.39,1.56008,0.0
3,11.56,1.56008,0.0
4,11.73,0.360015,0.0
5,11.9,1.32006,0.0
6,12.08,0.800036,0.0
7,12.26,0.960045,0.0
8,12.44,0.600026,0.0
9,12.62,0.720032,0.0


### DMA configuration
The DMA was operated at 20C and 940 hPa, with 4 L min-1 sheath flow, and 1 L min-1 sample flow. The DMA geometry is the TSI cylindrical long column, operated with a positive polarity power supply. The inversion is carried out to 6 charges. Transmission efficiency through the DMA is unity. The DMA is gridded to 120 bins and the grid is spanning the full range of the DMA's capability (10V to 10kV). Use setupSMPS instead of setupDMA if you want to use the SMPS transfer function.

In [16]:
t, p, lpm = 293.15, 940e2, 1.666e-5      # Temperature [K], Pressure [Pa], L min-1 to m3 s-1
r₁, r₂, l = 9.37e-3,1.961e-2,0.44369     # DMA geometry [m]
thisΛ = DMAconfig(t,p,1lpm,4lpm,r₁,r₂,l,0.0,:+,6,:cylindrical)  
#@time thisδ = setupSMPS(Λ, 10000, 10, 120, 1.0)     # Note that this is slower than setupDMA

@time thisδ = setupDMA(thisΛ, vtoz(thisΛ,10000), vtoz(thisΛ,10), 120);

  3.325591 seconds (32.48 M allocations: 856.557 MiB, 3.64% gc time)


<div class="alert alert-warning">

The @time macro benchmarks the grid creation. Most of the time is spent for precomputing the various inversion matrices. This is a one-time cost.
    
</div>

In [4]:
𝕣 = (df,:Dp,:Rcn,thisδ) |> interpolateDataFrameOntoδ;

<div class="alert alert-warning">

This function takes the columns :Dp and :Rcn from the DataFrame df and creates a response distribution of the type SizeDistribution that matches the DMA grid thisδ. 
    
</div>

In [17]:
𝐈, 𝐒, 𝐀, λ =  thisδ.𝐈, thisδ.𝐒, thisδ.𝐀, 0.5
@time 𝕟ⁱⁿᵛ¹ = (𝐀'𝐀 + λ^2𝐈)^(-1) * (𝐀'𝕣  + λ^2 * 𝐒^(-1)*𝕣);

  0.030522 seconds (65 allocations: 1.433 MiB)


<div class="alert alert-warning">

Regularized inversion with prescribed λ is very fast. 
    
</div>

In [19]:
@time 𝕟ⁱⁿᵛ² = rinv(𝕣.N, thisδ,λ₁=0.1,λ₂=1.0)

  5.494671 seconds (29.06 k allocations: 346.560 MiB, 0.34% gc time)


SizeDistribution(Any[], [663.8450668610888, 633.1437880511719, 604.0899449867386, 576.5907424761283, 550.558342125766, 525.9095988409856, 502.56582017116267, 480.45254567646487, 459.4993434977981, 439.63962143424715  …  14.913517427200246, 14.482624706775722, 14.06440605843314, 13.658475593554542, 13.264459942948104, 12.881997805409158, 12.51073951517362, 12.150346627327536, 11.80049152028906, 11.460857014528372], [648.2824969459034, 618.4168561813783, 590.1516165305455, 563.396495673634, 538.0660333108326, 514.0793394332707, 491.3598640345287, 469.8351854261576, 449.4368143866167, 430.1000115468786  …  15.133840693483398, 14.696462188153262, 14.271955540484456, 13.859928426683652, 13.460001275498547, 13.071806806965347, 12.69498959054076, 12.32920562165532, 11.974121915781371, 11.629416119162498], [0.047351239541361674, 0.04697444727919969, 0.04659037267925715, 0.04619979891584925, 0.04580359814799515, 0.045402715875213495, 0.04499815353613204, 0.044590950171455215, 0.0441821639439553

In [15]:
using Pkg
Pkg.status("DifferentialMobilityAnalyzers")

Status `~/.julia/environments/v1.4/Project.toml`
  [050a7be8] DifferentialMobilityAnalyzers v2.0.0 [`~/Documents/Research/DifferentialMobilityAnalyzers.jl`]


<div class="alert alert-warning">

Regularized inversion using the L-curve method is slow. This is a recurring cost.
    
</div>

<div class="alert alert-danger">

The function rinv performs the inversion using the internal matrices. It is straightforward to use a custom matrix in the inversion. The L-curve inversion is performed in these steps

- setup the system
- run the lcorner function which finds the corner along the L-curve
- compute the size distribution
- create a SizeDistribution object
    
**Let's perform an inversion on our distribution using a custom matrix.**
    
</div>

In [ ]:
# Transmission function for DMA without loss function - swap out functions for your own case
T(zˢ,k,thisΛ,thisδ) = thisδ.Ω(Λ,thisδ.Z,zˢ/k).*thisδ.Tc(k,thisδ.Dp)

# Transmission function for thisΛ, thisδ DMA and only using single charged particles
my𝐀 = (hcat(map(zˢ->Σ(k->T(zˢ,k,thisΛ,thisδ),1),thisδ.Z)...))'

setupRegularization(my𝐀,thisδ.𝐈,𝕣.N,inv(thisδ.𝐒)*𝕣.N) # setup the system of equations to regularize using my𝐀
λopt = lcorner(0.01,1.0;n=10,r=3)                    # compute the optimal λ using recursive algorithn
N = clean((reginv(λopt, r = :Nλ))[1])                # find the inverted size distribution 
𝕟ⁱⁿᵛ³ = SizeDistribution([],thisδ.De,thisδ.Dp,thisδ.ΔlnD,N./thisδ.ΔlnD,N,:regularized);

In [ ]:
include("Scripts/pretty_plot_data_inversion.jl")

**Figure.** Left: raw response function from data file. Right: results from the three inversions.Note that 𝕟ⁱⁿᵛ³ only considers singly charged particles, thus showing an overestimate.

### Summary

A short script is sufficient to invert raw data from a DMPS or SMPS measurement.

```julia
# Load a simple comma delimited text file
df = CSV.read("example_data.csv")

# Setup the DMA
t, p, lpm = 293.15, 940e2, 16.66e-5      # Temperature [K], Pressure [Pa], L min-1 to m3 s-1
r₁, r₂, l = 9.37e-3,1.961e-2,0.44369     # DMA geometry [m]
thisΛ = DMAconfig(t,p,1lpm,4lpm,r₁,r₂,l,0.0,:+,6,:cylindrical)  
thisδ = setupDMA(Λ, vtoz(Λ,10000), vtoz(Λ,10), 120)

# Interpolate the data onto the DMA grid
𝕣 = (df,:Dp,:Rcn,thisδ) |> interpolateDataFrameOntoδ

# Fast inversion with prescribed λ
𝐈, 𝐒, 𝐀, λ =  thisδ.𝐈, thisδ.𝐒, thisδ.𝐀, 0.5
𝕟ⁱⁿᵛ¹ = (𝐀'𝐀 + λ^2𝐈)^(-1) * (𝐀'𝕣  + λ^2 * 𝐒^(-1)*𝕣)

# Slow inversion using λopt from the L-curve method
𝕟ⁱⁿᵛ² = rinv(𝕣.N, thisδ,λ₁=0.1,λ₂=1.0)
```

This example is also in a clean notebook: "Inversion of Size Distribution From Data Example"


<div class="alert alert-success">

**You have met the following learning objective.**

- Apply the code to invert a DMA response function loaded from a data file.
    
</div>

# 6. Inverting Size-resolved CCN Measurements


Size resolved CCN measurements have a long history (e.g. Cruz and Pandis, 1997, Snider et al., 2006). Particles are typically dried, charge neutralized, and passed through the DMA. At the exit, the flow is split between a CPC that measures all particles and a CCN counter that measures particles that form cloud droplets at a water supersaturation set by the instrument. In this configuration the DMA voltage can either be stepped or continuously scanned. The ratio of CCN-to-CPC response function is used to determine the fraction of particles that activate at a given diameter. The diameter where 50% of the particles activate is taken to be the activation diameter. <br>

<img src="Figures/nbs7_f01.png" width="350">
<figcaption><b> Figure. </b> Schematic of instrument setup for size resolved CCN measurements.  </figcaption> 

The activation of particles into cloud droplets is proportional to the volume of solute in the particle. Therefore, larger multiply charged particles activate first. This leads to a bias in the inferred D50 diameter if the activated fraction is computed from the ratio of the raw response functions (Petters et al., 2007, 2009).

In [ ]:
# Load a simple comma delimited text file - file contains :Dp, :Rcn, :Rccn
# Note that this is the same DMA/size distribution defined by this DMA
df = CSV.read("example_data.csv")
𝕣ᶜⁿ = (df,:Dp,:Rcn,thisδ) |> interpolateDataFrameOntoδ        # CN response distribution
𝕣ᶜᶜⁿ = (df,:Dp,:Rccn,thisδ) |> interpolateDataFrameOntoδ;     # CCN response distribution

<div class="alert alert-danger">
    
Thresholding is performed for bins with low counts, which results in NaN and Inf in 𝕒𝕗. This is necessary because at low concentrations a bin may have zero or very low concentration, resulting in unrealistic or NaN/InF values in the activated fraction array. Thus if counts/concentration is below the threhold, the activated fraction is forced to one for large diameters and zero for small diameters. The noise threshold may need to be adjusted for different datasets.
    
</div>

In [ ]:
function threshold!(𝕟::SizeDistribution, c::Float64, n1::Float64, n2::Float64)
   N = 𝕟.N  
   S = 𝕟.S
   S[(N .<= c) .& (𝕟.Dp .> 150)] .= n2
   N[(N .<= c) .& (𝕟.Dp .> 150)] .= n2
   S[(N .<= c) .& (𝕟.Dp .< 150)] .= n1
   N[(N .<= c) .& (𝕟.Dp .< 150)] .= n1
   𝕟.N = N
end

threshold!(𝕣ᶜⁿ, 0.1, 0.1, 0.1)
threshold!(𝕣ᶜᶜⁿ, 0.1, 0.0, 0.1)

𝕒𝕗 = 𝕣ᶜᶜⁿ/𝕣ᶜⁿ;

In [ ]:
include("Scripts/pretty_plot_ccn_data.jl")

**Figure**. Left: Raw CN and CCN response functions. Right: activated fraction defined as the ratio of the response functions. The figure  shows  a shoulder of particles that appear to activating early. These are +2 charged particles transmitting through the DMA that influence of +2 the observed ratio 𝕣ᶜᶜⁿ./𝕣ᶜⁿ.

## 6.1 Modeling the Actived Fraction

The CCN transmission function is modelded using a cumulative Gauss integral <br>
<center> $T_{af} = \frac{1}{2}\left[1 + \mathrm{erf}\left(\frac{d-\mu}{\sigma}\right) \right] $ </center>
This function is applied to the mobility size distribution. Then the response function is computed. Empirically, activated fraction can be computed using from the ratio of size distributions and response functions. 

In [ ]:
# CCN activation unction 
#  - 𝕟 is a size distribution from which the diameter vector is used
#  - μ is the activation diameter
#  - σ is the spread
Taf(𝕟,μ,σ) = @. 0.5 * (1.0 + erf((𝕟.Dp - μ)./(sqrt(2.0σ))))


## 6.2 Fitting Model
There are two ways to fit the response data. Possibility one is to compute the activated fraction from the inverted data and then fit the ratio to the activation model to find the μ,σ that best describes the data. An alternative method is to compute the regularized inverse from the the CN response function and use it to predict the ratio of 𝕣ᶜⁿ./𝕣ᶜᶜⁿ. In this case the model is
<center> $𝕟^{\mathrm{cn}} = (\bf{A}^\rm{T}\bf{A} + \lambda^\rm{2} \bf{I})^\rm{-1}(\bf{A}^\rm{T} \rm{𝕣^{\mathrm{cn}}} - \lambda^2\bf{S}^{-1} \rm{𝕣^{\mathrm{cn}}})$ </center>
<center> $\left( \frac{𝕣^{\mathrm{cn}}}{𝕣^{\mathrm{cn}}} \right)_{\mathrm{model}} = \frac{\mathbf{A}[T_{\mu,\sigma}\; .*\; 𝕟^{\mathrm{cn}}]}{\mathbf{A}𝕟^{\mathrm{cn}}}$ </center>
This mirrors the approach in Petters et al. (2007).

The model + fit can be solved in few lines of code.
```julia
𝕟ᶜⁿ = (𝐀'𝐀 + λ^2𝐈)^(-1) * (𝐀'𝕣ᶜⁿ  + λ^2 * 𝐒^(-1)*𝕣ᶜⁿ)
model(x,p) = (δ.𝐀*(𝕟ᶜⁿ.N.*Taf(𝕟ᶜⁿ, p[1], p[2])))./(δ.𝐀*𝕟ᶜⁿ.N)
fit = curve_fit(model, δ.Dp, 𝕣ᶜᶜⁿ.N./𝕣ᶜⁿ.N, [65.0, 3.0])
```

<div class="alert alert-danger">
    
Note that least square fitting library is not setup to take the SizeDistribution data type, and thus the operation is performed on the number field of these objects. 
    
</div>

In [ ]:
𝐈, 𝐒, 𝐀, λ =  thisδ.𝐈, thisδ.𝐒, thisδ.𝐀, 0.5
𝕟ᶜⁿ = (𝐀'𝐀 + λ^2𝐈)^(-1) * (𝐀'𝕣ᶜⁿ  + λ^2 * 𝐒^(-1)*𝕣ᶜⁿ)
model(x,p) = (𝐀 * (𝕟ᶜⁿ.N .* Taf(𝕟ᶜⁿ, p[1], p[2])))./( 𝐀 * 𝕟ᶜⁿ.N)
fit = curve_fit(model, 𝕒𝕗.Dp, 𝕒𝕗.N, [65.0, 3.0])
Ax = fit.param
afmodel = model(thisδ.Dp, Ax)
DataFrame(μ=round(Ax[1],digits=1), σ=round(Ax[2],digits=1))

<div class="alert alert-warning">
    
The predicted activation diameter is 86.4 nm. 
    
</div>

In [ ]:
include("Scripts/pretty_plot_ccn_data_fit.jl")

**Figure**. Left: Raw CN and CCN response functions. Right: activated fraction. The afmodel accurately describes the +2 charge shoulder. 

<div class="alert alert-danger">
    
- The +2 charge shoulder can in some cases be large and interfere with the measurement
- It is also possible to fit Taf to 𝕟ᶜⁿ/𝕟ᶜᶜⁿ, where both distributions are inverted. This results in extra noise since two inversions are performed.
- The model is only very weakly sensitive to the choice of λ. For small λ, negative values might need filtering. Use of the L-curve is not needed in most cases.
- This example is also in a clean notebook: "Inversion of Size Resolved CCN Data From Data Example"
    
</div>

<div class="alert alert-success">

**You have met the following learning objective.**

- Apply the code to invert size resolved CCN data loaded from a data file.
    
</div>

# 7. Summary and Perspective

# 8. Epilogue

# 9. References


Cruz, C. N. and S. N. Pandis (1997). A study of the ability of pure secondary organic aerosol to act as cloud condensation nuclei. Atmospheric Environment 31, 2205–2214, DOI:10.1016/S1352-2310(97)00054-X.

Hansen, P. C. (2000) The L-Curve and its Use in the Numerical Treatment of Inverse Problems, in Computational Inverse Problems in Electrocardiology, ed. P. Johnston, Advances in Computational Bioengineering, 119-142, WIT Press. <br> 

Hagwood, C. (1999) The DMA Transfer Function with Brownian
Motion a Trajectory/Monte-Carlo Approach, Aerosol Science & Technology, 30:1, 40-61, DOI:10.1080/027868299304877.

Hinds, W. C. (1999) Aerosol Technology, Properties, Behavior, and Measurement of Airborne Particles, Second Edition, John Wiley & Sons, Inc.

Kandlikar, M., & Ramachandran, G. (1999) Inverse methods for analysing aerosol spectrometer measurements: A critical review. Journal of Aerosol Science, 30(4), 413-437, DOI:10.1016/S0021-8502(98)00066-4. <br>

Knutson, E. O. & K. T. Whitby (1975) Aerosol classification by electric mobility:
Apparatus, theory, and applications. J. Aerosol Sci. (6)443-451, DOI:10.1016/0021-8502(75)90060-9.

Park, D. Dutcher, M. Emery, J. Pagels, H. Sakurai, J. Scheckman, S. Qian, M. R. Stolzenburg, X. Wang, J. Yang & P. H. McMurry (2008) Tandem Measurements of Aerosol Properties—A Review of Mobility Techniques with Extensions, Aerosol Science and Technology, 42:10, 801-816, DOI:10.1080/02786820802339561.

Snider, J. R., M. D. Petters, P. Wechsler, and P. S. K. Liu, (2006), Supersaturation in the Wyoming CCN instrument, Journal Atmospheric Oceanic Technology, 23, 1323-1339, doi:10.1175/JTECH1916.1. 

Sipkens, T. A., Olfert, J. S., & Rogak, S. N. (2020). Inversion methods to determine two-dimensional aerosol mass-mobility distributions: A critical comparison of established methods. J. Aerosol Sci. 140, 105484.

Petters, M. D., A. J. Prenni, S. M. Kreidenweis, P. J. DeMott (2007) On measuring the critical diameter of cloud condensation nuclei using mobility selected aerosol, Aerosol Science & Technology, 41(10), 907-913, doi:10.1080/02786820701557214.

Petters, M. D., C. M. Carrico, S. M. Kreidenweis, A. J. Prenni, P. J. DeMott, J. R. Collett Jr., and H. Moosmüller (2009) Cloud condensation nucleation ability of biomass burning aerosol, Journal Geophysical Research, 114, D22205, doi:10.1029/2009JD012353.

Petters, M. D. (2018) A language to simplify computation of differential mobility analyzer response functions, Aerosol Science & Technology, 52:12, 1437-1451, DOI:10.1080/02786826.2018.1530724. 

Reineking A. & J. Porstendörfer (1986) Measurements of Particle Loss Functions in a Differential Mobility Analyzer (TSI, Model 3071) for Different Flow Rates, Aerosol Science and Technology, 5:4, 483-486, DOI:10.1080/02786828608959112

Rader, D.J. and P.H. McMurry (1986) Application of the tandem differential mobility analyzer to studies of droplet growth or evaporation,Journal of Aerosol Science, 17(5), 771-787,DOI:10.1016/0021-8502(86)90031-5.

Stolzenburg, M. R. & P. H. McMurry (2008) Equations Governing Single and Tandem DMA Configurations and a New Lognormal Approximation to the Transfer Function, Aerosol Science and Technology, 42:6, 421-432, DOI:10.1080/02786820802157823.

Talukdar, Suddha S. & Mark T. Swihart (2003) An Improved Data Inversion Program for Obtaining Aerosol Size Distributions from Scanning Differential Mobility Analyzer Data, Aerosol Science and Technology, 37:2, 145-161, DOI:10.1080/02786820300952. <br>

TSI Inc. (2009) Series 3080 Electrostatic Classifiers. Operation and Service Manual, P/N 1933792, Revision J March 2009. 

Twomey, S. (1963) On the numerical solution of Fredholm integral equations of the first kind by inversion of the linear system produced by quadrature, Journal of the ACM, 19(1963), 97–101, DOI:0.1145/321150.321157. <br>

Wang, S. C. and Flagan, R. C. (1990). Scanning electrical mobility spectrometer.
Aerosol Science and Technology, 13:2, 230–240.

Wright, T. P., C. Song, S. Sears & M. D. Petters (2016) Thermodynamic and kinetic behavior of glycerol aerosol, Aerosol Science and Technology, 50:12, 1385-1396, DOI:10.1080/02786826.2016.1245405.

Wiedensohler, A. (1988) An approximation of the bipolar charge distribution for particles in the submicron size range, Journal of Aerosol Science, 19:3, 387-389, DOI:10.1016/0021-8502(88)90278-9.

Xue, H., Moyle, A. M., Magee, N., Harrington, J. Y., and Lamb, D. (2005), Experimental Studies of Droplet Evaporation Kinetics: Validation of Models for Binary and Ternary Aqueous Solutions. J. Atmos. Sci., 62(12):4310–4326, DOI:10.1175/JAS3623.1.